In [1]:
import numpy as np
from pyscf import gto, scf, fci
from tabulate import tabulate
from RDMFS import HF_U2RDM, MU_U2RDM, GU_U2RDM, BBC1_U2RDM
from utils import spectral_clean, DM2_DiagSum, BST_FIDX, twoDM_Eigvals
import scipy 

In [2]:
# Get Hydrogen in mo basis from PySCF

mol = gto.Mole()
mol.unit = 'B' 
L = 1.5
mol.atom = f"""
# He 0 0 0 
H   {-L}   {-L}   0.0
H    {L}   {-L}   0.0
H    {L}    {L}   0.0
H   {-L}    {L}   0.0
"""
# this basis has 2 functions for Helium
# mol.basis = "ccpvdz"
mol.basis = "sto-3g"
# mol.basis = "631g"

mol.spin =  0
mol.verbose= 0
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri_ao = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

## Run Hartree-Fock.
mf = scf.UHF(mol)
mf.kernel()


# Harvest HF quantities

h1 = mf.get_hcore()
M = h1.shape[0]
C_a = mf.mo_coeff[0,:,:]
C_b = mf.mo_coeff[1,:,:]

h1_a = C_a.T@h1@C_a
h1_b = C_b.T@h1@C_b
h1_mo = (h1_a, h1_b)


h2_mo = BST_FIDX((eri_ao, eri_ao, eri_ao),S@C_a,S@C_b)

In [3]:
dm1_ao = mf.make_rdm1()
dm2_ao = mf.make_rdm2()

dm1_mo = dm1_ao.copy()
# # dm1_mo=C[:,0:N]@dm1_ao@C[:,0:N].T
dm1_mo[0]=(C_a.T@(S.T@dm1_mo[0]@S))@C_a
dm1_mo[1]=(C_a.T@(S.T@dm1_mo[1]@S))@C_a

dm2_mo = BST_FIDX(dm2_ao,S@C_a,S@C_b)

# P = C@C.T
# print(P)

occ_mo_a, occ_mo_b, C_mo_a, C_mo_b = spectral_clean(dm1_mo, h1.shape[0])
print(occ_mo_a, np.sum(occ_mo_a))
print(occ_mo_b, np.sum(occ_mo_b))


[1.00000000e+00 1.00000000e+00 1.79735152e-16 0.00000000e+00] 1.9999999999999993
[1.00000000e+00 1.00000000e+00 1.02571065e-16 0.00000000e+00] 1.9999999999999987


In [4]:
dm2_HF_mo = HF_U2RDM(occ_mo_a, occ_mo_b,M)
dm2_MU_mo = MU_U2RDM(occ_mo_a, occ_mo_b,M)
dm2_GU_mo = GU_U2RDM(occ_mo_a, occ_mo_b,M)
dm2_BBC1_mo = BBC1_U2RDM(occ_mo_a, occ_mo_b,M,mol.nelec[0],mol.nelec[1])


In [10]:
print(DM2_DiagSum(dm2_MU_mo), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum(dm2_HF_mo), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum(dm2_GU_mo), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum(dm2_BBC1_mo), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum(dm2_mo), scipy.special.binom(np.sum(mol.nelec),2))

11.999999999999984 6.0
11.999999999999988 6.0
11.999999999999988 6.0
11.999999999999984 6.0
11.999999999999984 6.0


In [11]:
# Run FCI  
cisolver = fci.direct_uhf.FCI()
cisolver.max_cycle = 100
cisolver.conv_tol = 1e-8
e_mo, ci_mo = cisolver.kernel(h1_mo, h2_mo, h1.shape[0], mol.nelec, ecore=mf.energy_nuc())
dm1_fci, dm2_fci  = cisolver.make_rdm12s(ci_mo, h1.shape[0], mol.nelec)
occ_mo_a, occ_mo_b, C_mo_a, C_mo_b = spectral_clean(dm1_fci, h1.shape[0])
print(occ_mo_a, np.sum(occ_mo_a))
print(occ_mo_b, np.sum(occ_mo_b))

[0.98295595 0.5        0.5        0.01704405] 2.000000000000001
[0.98295595 0.5        0.5        0.01704405] 2.0000000000000013


In [12]:
dm2_HF_fci = HF_U2RDM(occ_mo_a, occ_mo_b, mol.nao)
dm2_MU_fci = MU_U2RDM(occ_mo_a, occ_mo_b, mol.nao)
dm2_GU_fci = GU_U2RDM(occ_mo_a, occ_mo_b, mol.nao)
dm2_BBC1_fci = BBC1_U2RDM(occ_mo_a, occ_mo_b, mol.nao, mol.nelec[0], mol.nelec[1])

In [13]:
print(np.sum(twoDM_Eigvals(dm2_HF_fci)), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(dm2_MU_fci)), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(dm2_GU_fci)), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(dm2_BBC1_fci)), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(dm2_fci)), scipy.special.binom(4,2))

13.067014208521432 6.0
12.000000000000014 6.0
13.067014208521432 6.0
12.000000000000014 6.0
12.000000000000005 6.0


In [14]:
print(twoDM_Eigvals(dm2_HF_fci))

[-1.42194853e-16 -1.00613962e-16 -1.72974906e-17 -1.53376447e-17
 -1.04083409e-17 -1.41478297e-18  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.72220638e-18  6.09327227e-18
  8.67361738e-18  1.53376447e-17  2.14852215e-17  2.30825504e-17
  2.90499703e-04  2.90499703e-04  8.52202592e-03  8.52202592e-03
  8.52202592e-03  8.52202592e-03  8.52202592e-03  8.52202592e-03
  8.52202592e-03  8.52202592e-03  1.67535521e-02  1.67535521e-02
  1.67535521e-02  1.67535521e-02  1.70440518e-02  1.70440518e-02
  1.70440518e-02  1.70440518e-02  3.35071043e-02  3.35071043e-02
  2.50000000e-01  2.50000000e-01  2.50000000e-01  2.50000000e-01
  2.50000000e-01  2.50000000e-01  2.50000000e-01  2.50000000e-01
  4.91477974e-01  4.91477974e-01  4.91477974e-01  4.91477974e-01
  4.91477974e-01  4.91477974e-01  4.91477974e-01  4.91477974e-01
  5.00000000e-01  5.00000000e-01  9.66202396e-01  9.66202396e-01
  9.82955948e-01  9.82955